In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

26/01/26 02:53:17 WARN Utils: Your hostname, codespaces-f3c854 resolves to a loopback address: 127.0.0.1; using 10.0.0.189 instead (on interface eth0)
26/01/26 02:53:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


26/01/26 02:53:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Approach 1 (more troublesome) - Use pandas to create df, input into Spark to create Spark DF to infer schema, then manually create schema when creating new full Spark DF

In [5]:
import pandas as pd

In [6]:
!pwd

/workspaces/data-engineering-zoomcamp-homework/05-batch/code


In [7]:
df_green_pd = pd.read_csv('data/raw/green/2021/01/green_tripdata_2021_01.csv.gz', nrows=1000)

In [11]:
df_green_pd

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1,43,151,1,1.01,5.5,0.5,0.5,0.00,0.00,NaN,0.3,6.80,2,1,0.00
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1,166,239,1,2.53,10.0,0.5,0.5,2.81,0.00,NaN,0.3,16.86,1,1,2.75
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1,41,42,1,1.12,6.0,0.5,0.5,1.00,0.00,NaN,0.3,8.30,1,1,0.00
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1,168,75,1,1.99,8.0,0.5,0.5,0.00,0.00,NaN,0.3,9.30,2,1,0.00
4,2,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2,265,265,3,0.00,-52.0,0.0,-0.5,0.00,0.00,NaN,-0.3,-52.80,3,1,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2,2021-01-02 12:48:43,2021-01-02 13:54:15,N,1,15,177,1,25.76,77.0,0.0,0.5,2.75,12.24,NaN,0.3,92.79,1,1,0.00
996,1,2021-01-02 12:16:47,2021-01-02 12:24:56,N,1,116,74,1,2.00,9.0,0.0,0.5,1.95,0.00,NaN,0.3,11.75,1,1,0.00
997,2,2021-01-02 12:07:17,2021-01-02 12:12:06,N,1,42,41,1,0.87,5.5,0.0,0.5,0.00,0.00,NaN,0.3,6.30,2,1,0.00
998,2,2021-01-02 12:32:45,2021-01-02 12:59:46,N,1,82,218,1,11.88,35.0,0.0,0.5,0.00,0.00,NaN,0.3,35.80,2,1,0.00


In [8]:
from pyspark.sql import types

In [12]:
spark.createDataFrame(df_green_pd).schema

AttributeError: 'DataFrame' object has no attribute 'iteritems'

In [ ]:
# This error occurs because you're using pandas 2.0 or newer, where the iteritems() method has been removed. 9 PySpark 3.3.2 (which you're using) is not compatible with pandas 2.0+ as it still tries to call the deprecated iteritems() method when converting pandas DataFrames to Spark DataFrames. 7


In [18]:
green_schema = types.StructType([
    types.StructField("VendorID", types.IntegerType(), True),
    types.StructField("lpep_pickup_datetime", types.TimestampType(), True),
    types.StructField("lpep_dropoff_datetime", types.TimestampType(), True),
    types.StructField("store_and_fwd_flag", types.StringType(), True),
    types.StructField("RatecodeID", types.IntegerType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("passenger_count", types.IntegerType(), True),
    types.StructField("trip_distance", types.DoubleType(), True),
    types.StructField("fare_amount", types.DoubleType(), True),
    types.StructField("extra", types.DoubleType(), True),
    types.StructField("mta_tax", types.DoubleType(), True),
    types.StructField("tip_amount", types.DoubleType(), True),
    types.StructField("tolls_amount", types.DoubleType(), True),
    types.StructField("ehail_fee", types.DoubleType(), True),
    types.StructField("improvement_surcharge", types.DoubleType(), True),
    types.StructField("total_amount", types.DoubleType(), True),
    types.StructField("payment_type", types.IntegerType(), True),
    types.StructField("trip_type", types.IntegerType(), True),
    types.StructField("congestion_surcharge", types.DoubleType(), True)
])

yellow_schema = types.StructType([
    types.StructField("VendorID", types.IntegerType(), True),
    types.StructField("tpep_pickup_datetime", types.TimestampType(), True),
    types.StructField("tpep_dropoff_datetime", types.TimestampType(), True),
    types.StructField("passenger_count", types.IntegerType(), True),
    types.StructField("trip_distance", types.DoubleType(), True),
    types.StructField("RatecodeID", types.IntegerType(), True),
    types.StructField("store_and_fwd_flag", types.StringType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("payment_type", types.IntegerType(), True),
    types.StructField("fare_amount", types.DoubleType(), True),
    types.StructField("extra", types.DoubleType(), True),
    types.StructField("mta_tax", types.DoubleType(), True),
    types.StructField("tip_amount", types.DoubleType(), True),
    types.StructField("tolls_amount", types.DoubleType(), True),
    types.StructField("improvement_surcharge", types.DoubleType(), True),
    types.StructField("total_amount", types.DoubleType(), True),
    types.StructField("congestion_surcharge", types.DoubleType(), True)
])

In [ ]:
df_green = spark.read \
    .option("header", "true") \
    .schema(green_schema) \
    .csv('data/raw/green/2021/01/')

# [My approach / approach 2] DF of 1st 1001 rows - then inferSchema, now reuse schema to read full df

In [45]:
# Method 1: Use samplingRatio parameter
# Adjust based on your file size - samplingRatio 0.01 samples ~1% of data
#    the sample ratio affects the count of rows used for inferring
#    The first few 100 rows will be used if samplingRatio is None. 
#    https://spark.apache.org/docs/latest/api/python/_modules/pyspark/sql/session.html#SparkSession.createDataFrame
# SO NO MATTER OUR samplingRatio setting, even if it is lower than 0.01 (eg 0.000001), the count of records in our df will be the SAME!
df_green_a = spark.read \
    .option("header", "true") \
    .option("samplingRatio", 0.01) \
    .option("inferSchema", True) \
    .csv('data/raw/green/2021/01/')


# Method 2: Read just 1000 rows first to get schema, then apply to full dataset
df_1000_b = spark.read \
    .option("header", "true") \
    .option("inferSchema", True) \
    .csv('data/raw/green/2021/01/') \
    .limit(1000)


In [46]:
df_green_a.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('lpep_pickup_datetime', TimestampType(), True), StructField('lpep_dropoff_datetime', TimestampType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', IntegerType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', StringType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', IntegerType(), True), StructField('trip_type', IntegerType(), True), StructField('congestion_surcharge',

In [47]:
df_1000_b.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('lpep_pickup_datetime', TimestampType(), True), StructField('lpep_dropoff_datetime', TimestampType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', IntegerType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', StringType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', IntegerType(), True), StructField('trip_type', IntegerType(), True), StructField('congestion_surcharge',

In [48]:
df_green_a.count(), df_1000_b.count()

(76518, 1000)

In [49]:
schema = df_1000_b.schema

In [50]:
# now read the full df in 2021-01
df_green = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('data/raw/green/2021/01/')

In [51]:
df_green.count()

76518

# Schema for yellow taxi dataset

In [61]:
# approach 2 - method 1 - inferSchema from random 1% of dataset
df_yellow = spark.read \
    .option("header", "true") \
    .option("samplingRatio", 0.01) \
    .option("inferSchema", True) \
    .csv('data/raw/yellow/2021/01/')


[Stage 161:>                                                        (0 + 1) / 1]

In [62]:
df_yellow.count()

1369765

In [65]:
df_yellow.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampType(), True), StructField('tpep_dropoff_datetime', TimestampType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', IntegerType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', IntegerType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True)])

In [66]:
yellow_schema = df_yellow.schema

# Convert CSV to JSON, AND partition green and yellow taxi parquet datasets

In [67]:
green_schema = schema

In [69]:
green_schema

StructType([StructField('VendorID', IntegerType(), True), StructField('lpep_pickup_datetime', TimestampType(), True), StructField('lpep_dropoff_datetime', TimestampType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', IntegerType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', StringType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', IntegerType(), True), StructField('trip_type', IntegerType(), True), StructField('congestion_surcharge',

In [68]:
yellow_schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampType(), True), StructField('tpep_dropoff_datetime', TimestampType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', IntegerType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', IntegerType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True)])

In [56]:
year = 2020

for month in range(1, 13):
    print(f'processing data for {year}/{month}')

    # raw => CSV
    # pq => parquet
    input_path = f'data/raw/green/{year}/{month:02d}/'
    output_path = f'data/pq/green/{year}/{month:02d}/'

    df_green = spark.read \
        .option("header", "true") \
        .schema(green_schema) \
        .csv(input_path)

    # 4 cores, so create 4 partitions, from each CSV file to 4 parquet files each
    df_green \
        .repartition(4) \
        .write.parquet(output_path, mode="overwrite")

processing data for 2020/1


processing data for 2020/2


processing data for 2020/3


processing data for 2020/4
processing data for 2020/5
processing data for 2020/6
processing data for 2020/7
processing data for 2020/8
processing data for 2020/9
processing data for 2020/10
processing data for 2020/11
processing data for 2020/12


In [59]:
year = 2021 

for month in range(1, 8):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/green/{year}/{month:02d}/'
    output_path = f'data/pq/green/{year}/{month:02d}/'

    df_green = spark.read \
        .option("header", "true") \
        .schema(green_schema) \
        .csv(input_path)

    df_green \
        .repartition(4) \
        .write.parquet(output_path, mode="overwrite")

processing data for 2021/1
processing data for 2021/2
processing data for 2021/3
processing data for 2021/4
processing data for 2021/5
processing data for 2021/6


processing data for 2021/7


In [70]:
year = 2020

for month in range(1, 13):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/yellow/{year}/{month:02d}/'
    output_path = f'data/pq/yellow/{year}/{month:02d}/'

    df_yellow = spark.read \
        .option("header", "true") \
        .schema(yellow_schema) \
        .csv(input_path)

    df_yellow \
        .repartition(4) \
        .write.parquet(output_path, mode="overwrite")

processing data for 2020/1


processing data for 2020/2


processing data for 2020/3


processing data for 2020/4


processing data for 2020/5


...

processing data for 2020/8


processing data for 2020/9


processing data for 2020/10


processing data for 2020/11


processing data for 2020/12


In [71]:
year = 2021

for month in range(1, 8):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/yellow/{year}/{month:02d}/'
    output_path = f'data/pq/yellow/{year}/{month:02d}/'

    df_yellow = spark.read \
        .option("header", "true") \
        .schema(yellow_schema) \
        .csv(input_path)

    df_yellow \
        .repartition(4) \
        .write.parquet(output_path, mode="overwrite")

processing data for 2021/1


processing data for 2021/2


processing data for 2021/3


processing data for 2021/4


processing data for 2021/5


processing data for 2021/6


processing data for 2021/7
